## Some attempts at Temporal Convolutions

Network inspired by Schirrmeister - performing temporal convolutions

In [29]:
# Imports and setup

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import GRU
import matplotlib.pyplot as plt

from load_data import *
from utilities import *
from temporal_cnn import TEMPORAL_CNN


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Loading Data and Train/Test/Val Splitting

In [3]:
# Loads data from the EEG dataset and removes all EOG data

person_train_val, X_train_val, y_train_val, person_test, X_test, y_test = load_EEG_data()

print ('Training/Valid data shape: {}'.format(X_train_val.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_val.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_val.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
X_train, y_train, X_val, y_val = split_train_val(X_train_val, y_train_val, percent_validation=0.1)

print ('Training data shape: {}'.format(X_train.shape))
print ('Training target shape: {}'.format(y_train.shape))
print ('Validation data shape: {}'.format(X_val.shape))
print ('Validation target shape: {}'.format(y_val.shape))

Training data shape: (1904, 22, 1000)
Training target shape: (1904,)
Validation data shape: (211, 22, 1000)
Validation target shape: (211,)


### One Hot Encoding for Output Labels

In [41]:
print (y_val[33])
print (y_val.shape)
print (y_test.shape)

y_train_labels = one_hot_encode(y_train)
y_val_labels = one_hot_encode(y_val)
y_test_labels = one_hot_encode(y_test)

print (y_val_labels[33])
print (y_val_labels.shape)
print (y_test_labels.shape)


769
(211,)
(443,)
[1. 0. 0. 0.]
(211, 4)
(443, 4)


### Formatting for 2D convolution

In [0]:
N,E,T = X_train.shape
X_train_tmp = X_train.reshape((N,E,T,1))

N,E,T = X_val.shape
X_val_tmp = X_val.reshape((N,E,T,1))

N,E,T = X_test.shape
X_test_tmp = X_test.reshape((N,E,T,1))

### Temporal CNN Training and Evaluation

Now we test out our network - start with batchnorm and dropout

In [36]:
temp_cnn = TEMPORAL_CNN(X_train_tmp, y_train_labels, X_val_tmp, y_val_labels, X_test_tmp, y_test_labels,dropout=0.4,use_batchnorm=True)
temp_cnn.build_model()
temp_cnn.train(epochs=50)

test_loss, test_accuracy = temp_cnn.evaluate()

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Train on 1904 samples, validate on 211 samples
Epoch 1/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.4395 - acc: 0.3199 - val_loss: 1.3689 - val_acc: 0.4171
Epoch 2/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.2340 - acc: 0.4438 - val_loss: 1.2794 - val_acc: 0.4313
Epoch 3/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.1207 - acc: 0.5021 - val_loss: 1.2185 - val_acc: 0.4739
Epoch 4/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.0330 - acc: 0.5536 - val_loss: 1.1909 - val_acc: 0.5166
Epoch 5/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.8942 - acc: 0.6612 - val_loss: 1.2943 - val_acc: 0.4360
Epoch 6/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.8115 - acc: 0.6949 - val_loss: 1.2295 - val_acc: 0.4976
Epoch 7/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.7563 - acc: 0.7033 - val_loss: 1.0434 - val_acc: 0.5213
E

### Using Elu instead of ReLu

In [38]:
temp_cnn = TEMPORAL_CNN(X_train_tmp, y_train_labels, X_val_tmp, y_val_labels, X_test_tmp, y_test_labels,dropout=0.4,use_batchnorm=True,use_elu=True)
temp_cnn.build_model()
temp_cnn.train(epochs=50)

test_loss, test_accuracy = temp_cnn.evaluate()

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Train on 1904 samples, validate on 211 samples
Epoch 1/50
1904/1904 [==============================] - 6s 3ms/sample - loss: 1.3645 - acc: 0.3372 - val_loss: 1.2609 - val_acc: 0.4455
Epoch 2/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.2035 - acc: 0.4627 - val_loss: 1.1842 - val_acc: 0.4408
Epoch 3/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.0835 - acc: 0.5509 - val_loss: 1.0492 - val_acc: 0.5213
Epoch 4/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.9830 - acc: 0.5993 - val_loss: 1.1729 - val_acc: 0.5024
Epoch 5/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.9294 - acc: 0.6366 - val_loss: 1.0683 - val_acc: 0.5403
Epoch 6/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.8790 - acc: 0.6439 - val_loss: 1.0243 - val_acc: 0.5782
Epoch 7/50
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.8473 - acc: 0.6681 - val_loss: 1.0327 - val_acc: 0.5592
E

### Increasing dropout to combat overfitting

(Didn't seem to help)

In [40]:
temp_cnn = TEMPORAL_CNN(X_train_tmp, y_train_labels, X_val_tmp, y_val_labels, X_test_tmp, y_test_labels,dropout=0.55,use_batchnorm=True,use_elu=True)
temp_cnn.build_model()
temp_cnn.train(epochs=25)

test_loss, test_accuracy = temp_cnn.evaluate()

print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)

Train on 1904 samples, validate on 211 samples
Epoch 1/25
1904/1904 [==============================] - 6s 3ms/sample - loss: 1.3486 - acc: 0.3445 - val_loss: 1.3430 - val_acc: 0.4502
Epoch 2/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.1918 - acc: 0.4795 - val_loss: 1.2548 - val_acc: 0.4739
Epoch 3/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.0830 - acc: 0.5373 - val_loss: 1.1435 - val_acc: 0.4882
Epoch 4/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 1.0192 - acc: 0.5704 - val_loss: 1.1645 - val_acc: 0.4976
Epoch 5/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.9556 - acc: 0.6077 - val_loss: 1.1090 - val_acc: 0.5498
Epoch 6/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.9119 - acc: 0.6271 - val_loss: 1.0895 - val_acc: 0.5213
Epoch 7/25
1904/1904 [==============================] - 5s 3ms/sample - loss: 0.8723 - acc: 0.6376 - val_loss: 1.0024 - val_acc: 0.5735
E

### Result Summary:

**Best Test Accuracy:** 62% (better than reported 55%)